In [2]:
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
import json
import time


In [3]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
image_pred = pd.read_csv("image_predictions.tsv", sep='\t')

In [4]:
twitter_archive.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
image_pred.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [6]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [7]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [8]:
with open("tweet_json.txt") as file:
    twt_data = json.load(file)
tweet_info = pd.DataFrame(columns=["tweet_id", "timestamp", "retweet_count", "favorite_count"])
for twt in twt_data["tweets"]:
    tweet_info = tweet_info.append({"tweet_id":twt['id'], "timestamp":twt['created_at']\
                                   ,"retweet_count":twt['retweet_count'], "favorite_count":\
                                    twt['favorite_count']},ignore_index = True) 
    

In [9]:
tweet_info.head(5)

,tweet_id,timestamp,retweet_count,favorite_count
0,892420643555336193,Tue Aug 01 16:23:56 +0000 2017,8330,38100
1,892177421306343426,Tue Aug 01 00:17:27 +0000 2017,6154,32690
2,891815181378084864,Mon Jul 31 00:18:03 +0000 2017,4072,24599
3,891689557279858688,Sun Jul 30 15:58:51 +0000 2017,8474,41456
4,891327558926688256,Sat Jul 29 16:00:24 +0000 2017,9168,39628


In [10]:
tweet_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
tweet_id          2340 non-null object
timestamp         2340 non-null object
retweet_count     2340 non-null object
favorite_count    2340 non-null object
dtypes: object(4)
memory usage: 73.2+ KB


# Assess

## Quality Issues


#### `twitter_archive` table
<ul>
    <li>timestamp to date time data type</li>
    <li>retweeted_status_timestamp is an object rather than data_time data type.</li>
    <li>Symbols in text column</li>
    <li>vine.co instead of vine.com
    <li>tweet_id to be converted to string data type from integer</li>
    <li>in_reply_to_status_id to String data type</li>
    <li>in_reply_to_user_id to String data type</li>
    
</ul>

#### `image_pred` table
<ul>
    <li>"a" tags in source</li>    
    <li>"_" used as separator in p1 column of image_pred df</li>
    <li>"_" used as separator in p2 column of image_pred df</li>
    <li>"_" used as separator in p3 column of image_pred df</li>
    <li>tweet_id in image_pred to be converted to strint data type from integer</li>
    <li>Types of dogs to category data type</li>
    <li>image_num to String data type</li>
    <li>desktop_computer as value in p1</li>
</ul>

#### `tweet_info` table
<ul>
    <li>timestamp should be date time data type</li>
    <li>retweet_count column should be integer</li>
    <li>favorite_count should be integer</li>
</ul>

## Tidiness Issues

<ul>
    <li>Type of dog in "doggo", "floofer", "pupper", "puppo" columns instead of one column.</li>
    <li>`tweet_info` table should be a part of `twitter_archive` table</li>
</ul>

# Clean

In [11]:
twitter_clean = twitter_archive.copy()
image_pred_clean = image_pred.copy()


### `twitter_archive`: timestamp and retweeted_status_timestamp to date time data type

#### Define

timestamp column and retweeted_status_timestamp column is in object data type and changing it to date_time date type makes analysis easier

#### Code

In [12]:
twitter_clean["timestamp"] = pd.to_datetime(twitter_archive["timestamp"])
twitter_clean["retweeted_status_timestamp"] = pd.to_datetime(twitter_clean["retweeted_status_timestamp"])

#### Test

In [13]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

### `image_pred`: HTML a tags in source column

#### Define

URL links are enclosed inside opening and closing "a" tags.

#### Code

In [14]:
twitter_clean["source"] = twitter_archive["source"].str.extract(r'(http[^\"^\s]*)')

#### Test

In [38]:
twitter_clean.source.sample(10)

1920              http://twitter.com/download/iphone
1372              http://twitter.com/download/iphone
1343              http://twitter.com/download/iphone
1899              http://twitter.com/download/iphone
1625                                  http://vine.co
308               http://twitter.com/download/iphone
1599              http://twitter.com/download/iphone
477               http://twitter.com/download/iphone
1646              http://twitter.com/download/iphone
1723    https://about.twitter.com/products/tweetdeck
Name: source, dtype: object

'<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>'

In [40]:
tweet_info.head(5)

,tweet_id,timestamp,retweet_count,favorite_count
0,892420643555336193,Tue Aug 01 16:23:56 +0000 2017,8330,38100
1,892177421306343426,Tue Aug 01 00:17:27 +0000 2017,6154,32690
2,891815181378084864,Mon Jul 31 00:18:03 +0000 2017,4072,24599
3,891689557279858688,Sun Jul 30 15:58:51 +0000 2017,8474,41456
4,891327558926688256,Sat Jul 29 16:00:24 +0000 2017,9168,39628


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


### `tweet_info` should be a part of `twitter_archive`

#### Define

The columns "retweet_count" and "favorite_counts" should be a part of twitter_archive and these two can be merged on tweet_id

#### Code

In [58]:
tweet_info["tweet_id"] = tweet_info.tweet_id.astype(int)
twitter_clean = twitter_clean.merge(tweet_info, how="left", on="tweet_id")


#### Test

In [59]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 23 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp_x                   2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
timestamp_y     

### `twitter_archive` tweet_id should be a string

#### Define
Convert tweet_id to String

### Code

In [57]:
twitter_clean["tweet_id"] = twitter_clean.tweet_id.astype(str)

#### Test

In [51]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetim